In [6]:
#一共640题，看一天可以整理多少个

In [8]:
#SQL CheatSheet
#https://www.stratascratch.com/blog/sql-cheat-sheet-technical-concepts-for-the-job-interview/

# 1. Most Profitable Companies


Forbes
Medium
ID 10354


Find the 3 most profitable companies in the entire world.
Output the result along with the corresponding company name.
Sort the result based on profits in descending order.


# 2.Workers With The Highest Salaries


Interview Question Date: July 2021

Amazon
DoorDash
Easy
ID 10353


You have been asked to find the job titles of the highest-paid employees.


Your output should include the highest-paid title or multiple titles with the same salary.

DataFrames: worker, titleExpected Output Type: pandas.DataFrame

# 3. Users By Average Session Time


Interview Question Date: July 2021

Meta/Facebook
Medium
ID 10352

Calculate each user's average session time. A session is defined as the time difference between a page_load and page_exit. For simplicity, assume a user has only 1 session per day and if there are multiple of the same events on that day, consider only the latest page_load and earliest page_exit, with an obvious restriction that load time event should happen before exit time event . Output the user_id and their average session time.

Table: facebook_web_log

# 4.Activity Rank


Interview Question Date: July 2021

Google
Medium
ID 10351

Find the email activity rank for each user. Email activity rank is defined by the total number of emails sent. The user with the highest number of emails sent will have a rank of 1, and so on. Output the user, total emails, and their activity rank. Order records by the total emails in descending order. Sort users with the same number of emails in alphabetical order.
In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails. For tie breaker use alphabetical order of the user usernames.

Table: google_gmail_emails

# 5.Algorithm Performance


Interview Question Date: July 2021

Meta/Facebook
Hard


Meta/Facebook is developing a search algorithm that will allow users to search through their post history. You have been assigned to evaluate the performance of this algorithm.


We have a table with the user's search term, search result positions, and whether or not the user clicked on the search result.


Write a query that assigns ratings to the searches in the following way:
•	If the search was not clicked for any term, assign the search with rating=1
•	If the search was clicked but the top position of clicked terms was outside the top 3 positions, assign the search a rating=2
•	If the search was clicked and the top position of a clicked term was in the top 3 positions, assign the search a rating=3


As a search ID can contain more than one search term, select the highest rating for that search ID. Output the search ID and its highest rating.


Example: The search_id 1 was clicked (clicked = 1) and its position is outside of the top 3 positions (search_results_position = 5), therefore its rating is 2.

Table: fb_search_events

In [ ]:
--法2：
SELECT 
    search_id,
    MAX(CASE WHEN clicked = 0 THEN 1
        WHEN search_results_position > 3 THEN 2
        ELSE 3 END) as max_rating
FROM fb_search_events
GROUP BY search_id

# 6.Distances Traveled


Interview Question Date: December 2020

Lyft
Medium
ID 10324

Find the top 10 users that have traveled the greatest distance. Output their id, name and a total distance traveled.

Tables: lyft_rides_log, lyft_users

# 7.Finding User Purchases【有个video可看】


Interview Question Date: December 2020

Amazon
Medium
ID 10322

Write a query that'll identify returning active users. A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. Output a list of user_ids of these returning active users.

Table: amazon_transactions

In [ ]:
--法4：lag建立新的一列，指前面变量的前一个值，再用where filter 相减 <=7即可
--lag(X, 1) over(partition by Y order by X ASC) as previous【注意也是asc】

WITH previous_transaction AS(
SELECT
    user_id,
    created_at,
    LAG(created_at, 1) OVER(PARTITION BY user_id ORDER BY created_at ASC) AS previous_transaction
FROM amazon_transactions
)
SELECT 
  DISTINCT user_id
FROM previous_transaction
WHERE created_at - previous_transaction <= 7